In [1]:
import math
from collections import Counter
#香农公式计算信息熵
def calcShannonEnt(dataSet):
    casenum = len(dataSet)
    caselabel = [data[-1] for data in dataSet] #从数据集中得到类别标签
    #得到类别为key，出现次数value的字典
    labelCounts = Counter(caselabel)  
    infoEntropy = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/casenum
        infoEntropy -= prob*math.log(prob, 2)
    return infoEntropy

def CreateDataSet():
    dataset = res.values.tolist()
    labels = ['outlook', 'temperature', 'humidity', 'wind']
    attrivalues = {}
    for i in range(len(labels)):
        featValues = [example[i] for example in dataset]
        uniqueVals = set(featValues)
        attrivalues[labels[i]] = list(uniqueVals)
        return dataset, labels,attrivalues

#划分数据集：得到样本子集
def splitDataSet(dataSet, axis, value):
    subDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:#提取所有特征featVec[axis]的值为value的样本
            subFeatVec = featVec[:axis]
            subFeatVec.extend(featVec[axis+1:])
            subDataSet.append(subFeatVec)
    return subDataSet

#选取信息增益最大的属性作为节点
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0])-1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    #外循环：遍历数据集各列，计算最优特征
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy =0.0#初始化当前特征的熵
        for value in uniqueVals:#遍历特征的所有值
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        #print("{3}:{0}={1}-{2}".format(infoGain,baseEntropy,newEntropy,attrinames[i]))
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
         
    return bestFeature,bestInfoGain

#对于属性已经用完，仍然没有分类的情况，采用少数服从多数的方法 
def majorityCnt(classList):
    classCount = dict([classList.count(i),i] for i in classList)
    
    return classCount[max(classCount.keys())]
 

#创建树
def createTree(dataSet, attrinames,attrivalues):  
    classList = [example[-1] for example in dataSet]#创建需要创建树的训练数据的结果列表（例如最外层的列表是[N, N, Y, Y, Y, N, Y]）
    if classList.count(classList[0]) == len(classList):#如果所有的训练数据都是属于一个类别，则返回该类别
        return classList[0]  
    if (len(dataSet[0]) == 1):  #训练数据只给出类别数据（没给任何属性值数据），返回出现次数最多的分类名称
        return majorityCnt(classList)
    bestFeat,bestInfoGain = chooseBestFeatureToSplit(dataSet)   #选择信息增益最大的属性进行分（返回值是属性类型列表的下标）
    if bestInfoGain<0.01:
        return majorityCnt(classList)
    bestFeatLabel = attrinames[bestFeat]  #根据下表找属性名称当树的根节点
    myTree = {bestFeatLabel:{}}  #以bestFeatLabel为根节点建一个空树
    del(attrinames[bestFeat])  #从属性列表中删掉已经被选出来当根节点的属性
    featValues = attrivalues[bestFeatLabel]  #找出该属性所有训练数据的值（创建列表）
    uniqueVals = set(featValues)  #求出该属性的所有值得集合（集合的元素不能重复）
    for value in uniqueVals:  #根据该属性的值求树的各个分支
        subLabels = attrinames[:]
        subDataset = splitDataSet(dataSet, bestFeat, value)
        if len(subDataset)== 0:
            myTree[bestFeatLabel][value]= majorityCnt(classList) 
        else:
            myTree[bestFeatLabel][value] = createTree(subDataset,subLabels,attrivalues)  #根据各个分支递归创建树
    return myTree  #生成的树
          
          
myDat,attrinames,attrivalues = CreateDataSet()
tree = createTree(myDat,attrinames,attrivalues)#创建树
 
print (tree)

NameError: name 'res' is not defined